##### Parameters
- workspaceName - The name of the workspace where the source warehouse exists
- warehouseName - The name of the warehouse that is to be backed up
- workspaceBackupName - The name of the workspace that the lakehouse for backups exists
- lakehouseBackupName - The name of the lakehouse used for back ups
- backupDateTime - The datetime that the backup took place. Get this value from the backup lakehouse folder.

In [ ]:
workspaceName = 'WS_Demo_InternetSales'
warehouseName = 'WH_InternetSales'

workspaceBackupName = 'WS_Demo_InternetSales_Backup'
lakehouseBackupName = 'LH_DW_Backups'
backupDatetime = '20241210_1450'

In [ ]:
# https://www.rakirahman.me/directory-recursion-synapse/

def deep_ls(path: str, max_depth=1):
    """
    List all files and folders in specified path and
    subfolders within maximum recursion depth.
    """

    # List all files in path
    li = mssparkutils.fs.ls(path)

    # Return all files
    for x in li:
        if x.size != 0:
            yield x

    # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x

fileList = deep_ls(f'abfss://{workspaceBackupName}@onelake.dfs.fabric.microsoft.com/{lakehouseBackupName}.Lakehouse/Files/{workspaceName}/{warehouseName}/{backupDatetime}', max_depth=2)

tableList = []
for file in fileList:
    schema = file.path.split('/')[-2]
    table = file.name
    print(f'{schema = }, {table = }')
    tableList.append({"schema": schema, "table": table})

In [ ]:
from notebookutils import mssparkutils
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
        }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces', headers=header)
for workspace in response.json().get('value'):
  if workspace.get('displayName') == workspaceName:
    workspaceId = workspace.get('id')
  elif workspace.get('displayName') == workspaceBackupName:
    workspaceBackupId = workspace.get('id')

print(f'{workspaceId = }\n{workspaceBackupId = }')

##### Create temporary restore lakehouse in the workspace where the warehouse exists. "LH_temp_restore_{warehouseName}_{backupDateTime}

In [ ]:
lakehouseTempRestoreName = f"LH_temp_restore_{warehouseName}_{backupDatetime}"

body = {
    "displayName": lakehouseTempRestoreName
}

response = requests.request(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses', headers=header, data=json.dumps(body))
print(response.status_code)
print(response.json())


##### Get The lakehouse ids

In [ ]:
from notebookutils import mssparkutils
import requests, json

header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
        }

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceBackupId}/lakehouses', headers=header)
for item in response.json().get('value'):
  if item.get('displayName') == lakehouseBackupName:
    lakehouseBackupId = item.get('id')

response = requests.request(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses', headers=header)
for item in response.json().get('value'):
    lakehouseTempRestoreId = item.get('id')

print(f'{lakehouseBackupId = }\n{lakehouseTempRestoreId = }')

In [ ]:
from notebookutils import mssparkutils
import requests, json


workspaceIdTarget = workspaceId
itemIdTarget = lakehouseTempRestoreId

workspaceIdSource = workspaceBackupId
itemIdSource = lakehouseBackupId


url = f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceIdTarget}/items/{itemIdTarget}/shortcuts' #?shortcutConflictPolicy=GenerateUniqueName'

for table in tableList:
    body = {
        "name": f"{table.get('schema')}_{table.get('table')}"
        ,"path": "Tables"
        ,"target": {
            "oneLake": {
                "itemId": itemIdSource
                ,"path": f"Files/{workspaceName}/{warehouseName}/{backupDatetime}/{table.get('schema')}/{table.get('table')}"
                ,"workspaceId": workspaceIdSource
            }
        }
    }
    
    response = requests.request(method='post', url=url, headers=header, data=json.dumps(body))
    print(response.status_code)
    print(f'{response.status_code} - {response.json()}')

In [ ]:
import pyodbc, struct, itertools, time, datetime, re, uuid, json

fabricDWServer = 'n52dzjnhphme5jslxpytuo62ni-bwqqnh23wdjude66jg7pd6kctu.datawarehouse.fabric.microsoft.com'

connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={fabricDWServer};Database={workspaceName}'

# Use the credentials of the user executing the notebook
token = bytes(mssparkutils.credentials.getToken('pbi'), "UTF-8")
encoded_bytes = bytes(itertools.chain.from_iterable(zip(token, itertools.repeat(0))))
tokenstruct = struct.pack("<i", len(encoded_bytes)) + encoded_bytes

def get_result_set(cursor):
    if cursor.description:
        resultList = cursor.fetchall()
        resultColumns = columns = [column[0] for column in cursor.description]
    else:
        resultList = []
        resultColumns = []
    return [dict(zip(resultColumns, [str(col) for col in row])) for row in resultList]

with pyodbc.connect(connectionString, attrs_before = { 1256:tokenstruct }) as conn:
    for table in tableList:
        with conn.cursor() as cursor:
            queryStatement = f'TRUNCATE TABLE {warehouseName}.{table.get("schema")}.{table.get("table")}; INSERT INTO {warehouseName}.{table.get("schema")}.{table.get("table")} SELECT * FROM {lakehouseTempRestoreName}.dbo.{table.get("schema")}_{table.get("table")}'
            print(queryStatement)

            cursor.execute(queryStatement)
            
            queryMessage = str(cursor.messages) if cursor.messages else ""
            print(queryMessage)
            
            resultSet = get_result_set(cursor)
            print(resultSet)

            cursor.commit()


##### Drop the temporary lakehouse

In [ ]:
header = {'Authorization': f'Bearer {mssparkutils.credentials.getToken("pbi")}'
          ,"Content-Type": "application/json"
        }
        
response = requests.request(method='delete', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/lakehouses/{lakehouseTempRestoreId}', headers=header)

print(response.status_code)
print(response.text)